***notebook này tập trung vào việc sử dụng kĩ thuật adaptive synthetic sampling để giải quết vấn đề dữ liệu bọ mất cân bằng***

# Tổng quan về kĩ thuật adaptive synthetic sampling (adasyn technique)

Adaptive Synthetic Sampling (ADASYN) là một kỹ thuật xử lý mất cân bằng dữ liệu trong machine learning. Khi làm việc với dữ liệu có sự mất cân bằng, các mô hình học máy có xu hướng thiên vị về phía lớp chiếm đa số. ADASYN giúp giải quyết vấn đề này bằng cách tạo ra các mẫu tổng hợp cho lớp thiểu số.

Dưới đây là tất cả thông tin về ADASYN:

### Mục tiêu
- Tăng cường tính cân bằng của dữ liệu bằng cách tạo ra các mẫu tổng hợp cho lớp thiểu số.
- Tăng cường độ chính xác của các mô hình học máy trong việc dự đoán lớp thiểu số.

### Cách thức hoạt động
ADASYN dựa trên nguyên tắc sau:

1. **Xác định số lượng mẫu cần tổng hợp:** 
   - Xác định tỷ lệ mất cân bằng của dữ liệu.
   - Xác định số lượng mẫu cần tạo cho lớp thiểu số để đạt được sự cân bằng mong muốn.

2. **Tính toán độ phức tạp của các điểm dữ liệu thiểu số:**
   - Sử dụng phương pháp K-Nearest Neighbors (KNN) để tìm các điểm lân cận gần nhất cho mỗi điểm dữ liệu thiểu số.
   - Tính toán độ phức tạp của mỗi điểm dữ liệu thiểu số dựa trên số lượng điểm thuộc lớp đa số trong các điểm lân cận của nó.

3. **Tạo mẫu tổng hợp:**
   - Các điểm dữ liệu thiểu số có độ phức tạp cao hơn sẽ được ưu tiên tạo mẫu tổng hợp.
   - Sử dụng phương pháp nội suy để tạo ra các mẫu tổng hợp giữa các điểm dữ liệu thiểu số và các điểm lân cận của nó.

### Công thức
Để hiểu rõ hơn, đây là một số công thức chính trong ADASYN:

1. **Độ phức tạp \( r_i \) của mỗi điểm dữ liệu thiểu số \( x_i \):**
\[ r_i = \frac{\Delta_i}{k} \]
   - \( \Delta_i \) là số lượng các điểm thuộc lớp đa số trong các điểm lân cận của \( x_i \).
   - \( k \) là số lượng điểm lân cận được chọn.

2. **Số lượng mẫu tổng hợp \( G \) cần tạo:**
\[ G = (\text{tỷ lệ cân bằng mong muốn} - \text{tỷ lệ hiện tại}) \times \text{số lượng điểm lớp đa số} \]

3. **Phân bổ số lượng mẫu tổng hợp cho mỗi điểm dữ liệu thiểu số \( x_i \):**
\[ g_i = r_i \times G \]

4. **Tạo mẫu tổng hợp \( x_{\text{new}} \):**
\[ x_{\text{new}} = x_i + \theta \times (x_{zi} - x_i) \]
   - \( x_{zi} \) là một trong các điểm lân cận của \( x_i \).
   - \( \theta \) là một số ngẫu nhiên trong khoảng [0, 1].

### Ưu điểm
- **Tăng cường khả năng học của mô hình:** ADASYN giúp mô hình học máy học tốt hơn từ các mẫu thiểu số bằng cách tạo ra các mẫu tổng hợp gần với biên giới phân loại.
- **Linh hoạt:** ADASYN không yêu cầu cấu hình phức tạp và có thể dễ dàng tích hợp vào các pipeline học máy.

### Nhược điểm
- **Tăng thời gian tính toán:** Quá trình tính toán độ phức tạp và tạo mẫu tổng hợp có thể tốn thời gian khi dữ liệu lớn.
- **Khả năng gây overfitting:** Việc tạo ra quá nhiều mẫu tổng hợp có thể dẫn đến overfitting nếu không được kiểm soát.

### Ứng dụng
- **Phát hiện gian lận:** ADASYN được sử dụng để phát hiện các giao dịch gian lận trong các hệ thống tài chính.
- **Chẩn đoán y khoa:** Sử dụng để cải thiện độ chính xác của các hệ thống chẩn đoán bệnh hiếm gặp.
- **Nhận dạng hình ảnh:** Áp dụng trong các bài toán nhận dạng hình ảnh với sự mất cân bằng trong các lớp đối tượng.

### Mã nguồn Python
Dưới đây là một ví dụ về cách sử dụng ADASYN trong Python bằng thư viện `imbalanced-learn`:

```python
from imblearn.over_sampling import ADASYN
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Tạo dữ liệu giả
X, y = make_classification(n_samples=1000, n_features=20, n_informative=2, n_redundant=10, n_clusters_per_class=1, weights=[0.9, 0.1], flip_y=0, random_state=1)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Áp dụng ADASYN
adasyn = ADASYN(random_state=42)
X_resampled, y_resampled = adasyn.fit_resample(X_train, y_train)

# Huấn luyện mô hình
clf = RandomForestClassifier(random_state=42)
clf.fit(X_resampled, y_resampled)

# Dự đoán và đánh giá
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
```

### Kết luận
ADASYN là một kỹ thuật mạnh mẽ để giải quyết vấn đề mất cân bằng dữ liệu trong machine learning. Bằng cách tạo ra các mẫu tổng hợp từ các điểm dữ liệu thiểu số, ADASYN giúp cải thiện hiệu suất của các mô hình học máy trong việc dự đoán các lớp thiểu số.

# Thực hành

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from imblearn.over_sampling import ADASYN
from sklearn.ensemble import RandomForestClassifier


In [2]:
data_path = "E:/IAD/INTERN/01_diabetes/data/diabetes.csv"
df = pd.read_csv(data_path)
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
x = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

adasyn = ADASYN(random_state=42)
x_res, y_res = adasyn.fit_resample(x_train, y_train)

rfc = RandomForestClassifier(random_state=42)
rfc.fit(x_res, y_res)

y_pred = rfc.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'accuracy = {accuracy}')
print(f'precision = {precision}')
print(f'recall = {recall}')
print(f'f1 = {f1}')

accuracy = 0.7467532467532467
precision = 0.6212121212121212
recall = 0.7454545454545455
f1 = 0.6776859504132231
